In [1]:
import time
import taskingai
# Load TaskingAI API Key fron environment variable

# TaskingAI: Chat with Assistant Example

In this example, we will first create an assistant who knows the meaning of various numbers and will explain it in certain language.
Then we will start a chat with the assistant.

##  Create Assistant

In [2]:
from taskingai.tool import Action, ActionAuthentication, ActionAuthenticationType
from typing import List

# create an assistant action
NUMBERS_API_SCHEMA = {
        "openapi": "3.0.0",
        "info": {
            "title": "Numbers API",
            "version": "1.0.0",
            "description": "API for fetching interesting number facts"
        },
        "servers": [
            {
            "url": "http://numbersapi.com"
            }
        ],
        "paths": {
            "/{number}": {
            "get": {
                "summary": "Get fact about a number",
                "parameters": [
                {
                    "name": "number",
                    "in": "path",
                    "required": True,
                    "description": "The number to get the fact for",
                    "schema": {
                    "type": "integer"
                    }
                }
                ],
                "responses": {
                "200": {
                    "description": "A fact about the number",
                    "content": {
                    "text/plain": {
                        "schema": {
                        "type": "string"
                        }
                    }
                    }
                }
                }
            }
        }
    }
}
actions: List[Action] = taskingai.tool.bulk_create_actions(
    schema=NUMBERS_API_SCHEMA,
    authentication=ActionAuthentication(
        type=ActionAuthenticationType.none,
    )
)
action = actions[0]
print(f"created action: {action}\n")

created action: object='Action' action_id='bFBd4I6YoHOiIgEnTpICAgC8' schema={'info': {'title': 'Numbers API', 'version': '1.0.0', 'description': 'API for fetching interesting number facts'}, 'paths': {'/{number}': {'get': {'summary': 'Get fact about a number', 'responses': {'200': {'content': {'text/plain': {'schema': {'type': 'string'}}}, 'description': 'A fact about the number'}}, 'parameters': [{'in': 'path', 'name': 'number', 'schema': {'type': 'integer'}, 'required': True, 'description': 'The number to get the fact for'}]}}}, 'openapi': '3.0.0', 'servers': [{'url': 'http://numbersapi.com'}], 'security': [], 'components': {}} authentication=ActionAuthentication(type=<ActionAuthenticationType.none: 'none'>, secret=None, content=None) created_timestamp=1701345415792


In [3]:
from taskingai.assistant import Assistant, Chat, AssistantTool, AssistantToolType

# choose an available chat_completion model from your project
model_id = "Gk1145Bl"

assistant: Assistant = taskingai.assistant.create_assistant(
    model_id=model_id,
    name="My Assistant",
    description="A assistant who knows the meaning of various numbers.",
    system_prompt_template=[
        "You know the meaning of various numbers.",
        "No matter what the user's language is, you will use the {{langugae}} to explain."
    ],
    tools=[AssistantTool(
        type=AssistantToolType.action,
        id=action.action_id,
    )],
    retrievals=[],
    metadata={"foo": "bar"},
)
print(f"created assistant: {assistant}\n")

created assistant: object='Assistant' assistant_id='X5lMUSFLWzESDvTmy0SwOx0v' model_id='Gk1145Bl' name='My Assistant' description='A assistant who knows the meaning of various numbers.' system_prompt_template=['You know the meaning of various numbers.', "No matter what the user's language is, you will use the {{langugae}} to explain."] tools=[AssistantTool(type=<AssistantToolType.action: 'action'>, id='bFBd4I6YoHOiIgEnTpICAgC8')] retrievals=[] metadata={'foo': 'bar'} created_timestamp=1701345417694


## Start a Chat 

In [4]:
chat: Chat = taskingai.assistant.create_chat(
    assistant_id=assistant.assistant_id,
)
print(f"created chat: {chat.chat_id}\n")

created chat: SdELqszTR3IXt51rBj0svkSq


In [5]:
from taskingai.assistant import Message, MessageChunk

user_input = input("User Input: ")
while user_input != "q":
    # create user message
    taskingai.assistant.create_user_message(
        assistant_id=assistant.assistant_id,
        chat_id=chat.chat_id,
        text=user_input,
    )
    print(f"User: {user_input}")
    
    # generate assistant response
    assistant_message: Message = taskingai.assistant.generate_assistant_message(
        assistant_id=assistant.assistant_id,
        chat_id=chat.chat_id,
        system_prompt_variables={
            "language": "English"
        }
    )
    print(f"Assistant: {assistant_message.content.text}")
    time.sleep(2)
    user_input = input("User: ")

User: hi
Assistant: Hello! How can I assist you today?
User: what is the meaning of 12
Assistant: The number 12 is often associated with various meanings depending on the context. Here are some interpretations:

1. Numerology: In numerology, 12 is considered a powerful number associated with completion, harmony, and a sense of wholeness. It is often seen as a symbol of perfection, balance, and divine guidance.

2. Time: There are 12 months in a year, and this is based on the approximate time it takes for the moon to complete its cycle around the Earth. Additionally, there are 12 hours on a clock face, both in the morning and in the evening.

3. Mathematics: 12 is divisible by 1, 2, 3, 4, 6, and itself, making it a highly composite number. It also has a prime factorization of 2^2 × 3, which is significant in number theory.

4. Religion and Mythology: In many religious beliefs and mythologies, there are groups or sets of 12. For instance, there are 12 disciples of Jesus in Christianity, 

In [6]:
# response in stream

user_input = input("User Input: ")
while user_input != "q":
    # create user message
    taskingai.assistant.create_user_message(
        assistant_id=assistant.assistant_id,
        chat_id=chat.chat_id,
        text=user_input,
    )
    print(f"User: {user_input}")
    
    # generate assistant response
    assistant_message_response = taskingai.assistant.generate_assistant_message(
        assistant_id=assistant.assistant_id,
        chat_id=chat.chat_id,
        system_prompt_variables={
            "language": "English"
        },
        stream=True,
    )
    
    print(f"Assistant:", end=" ", flush=True)
    for item in assistant_message_response:
        
        if isinstance(item, MessageChunk):
            print(item.delta, end="", flush=True)
            
        elif isinstance(item, Message):
            print(f"\nmessage_id: {item.message_id}")
            
            
    time.sleep(2)
    user_input = input("User: ")

User: what about 127
Assistant: The number 127 is the atomic number of Unbiseptium, an element that has not yet been discovered. In the periodic table of elements, each unique element has an atomic number that corresponds to the number of protons in its nucleus. However, as of now, no element with 127 protons has been confirmed or named. Therefore, 127 represents a theoretical element that scientists have not yet observed or identified.
message_id: Mah1zSgBZpDuLvpEnVhEmWZy
User: bye
Assistant: Goodbye! If you have any more questions in the future, feel free to ask. Have a great day!
message_id: Mah1uIUcePTL7gvPjzl6JtpM


In [8]:
# list messages
messages = taskingai.assistant.list_messages(
    assistant_id=assistant.assistant_id,
    chat_id=chat.chat_id,
)
for message in messages:
    print(f"{message.role}: {message.content.text}")
    print("-"*100)

assistant: Goodbye! If you have any more questions in the future, feel free to ask. Have a great day!
----------------------------------------------------------------------------------------------------
user: bye
----------------------------------------------------------------------------------------------------
assistant: The number 127 is the atomic number of Unbiseptium, an element that has not yet been discovered. In the periodic table of elements, each unique element has an atomic number that corresponds to the number of protons in its nucleus. However, as of now, no element with 127 protons has been confirmed or named. Therefore, 127 represents a theoretical element that scientists have not yet observed or identified.
----------------------------------------------------------------------------------------------------
user: what about 127
----------------------------------------------------------------------------------------------------
assistant: You're welcome! If you have any 

In [9]:
# delete assistant
taskingai.assistant.delete_assistant(
    assistant_id=assistant.assistant_id,
)